In [2]:
import pickle
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level

loaded_model = pickle.load(open('../models/inflow_forecast_model_with_weather.pkl', 'rb'))


In [36]:

target_column = 'Total Inflow hrly'
df = pd.read_csv('https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/since-2020.csv')
df.ds = pd.to_datetime(df.ds)
df = df.drop(['Date', 'Time'], axis=1)
df = df.sort_values(by='ds', ascending=True)
df.rename(columns = {target_column:'y', 'Adm. requests cum':"Adm requests cum", 'Pts.waiting for admission CUM':'Pts waiting for admission CUM'}, inplace = True)
df.dropna(inplace=True)
df = df.reset_index(drop=True)
regressors = df.columns.tolist()
regressors.remove('y')
regressors.remove('ds')
df


,Stretcher Pts hrly,Stretcher Pts cum,Ambulatory Pts hrly,Ambulatory Pts cum,y,Total Inflow cum,Ambulances hrly,Ambulances cum,FLS hrly,Adm requests cum,...,GARAGE patient TBS,Consults > 2h in Vertical Except IM,Consult for IM >4h in Vertical,Plain films reqs > 2 hr in Vertical,CTs reqs > 2 hrs in Vertical,Psych Stretcher Pts1pt,Psych pts waiting for admission,ds,Total Pod TBS,Total Vertical TBS
0,9,108,5,142,14,250,6,39,0,27,...,1.0,1,0,0,3,0,0,2021-07-26 19:00:00,10,18.0
1,5,113,6,148,11,261,2,41,0,33,...,1.0,1,0,0,3,1,0,2021-07-26 20:00:00,12,9.0
2,5,118,2,150,7,268,1,42,0,34,...,0.0,1,0,0,4,4,0,2021-07-26 21:00:00,8,4.0
3,5,123,8,158,13,281,2,44,0,34,...,0.0,1,0,0,3,5,0,2021-07-26 22:00:00,9,6.0
4,3,126,7,165,10,291,3,47,0,37,...,1.0,0,0,1,3,6,0,2021-07-26 23:00:00,10,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7699,17,54,8,36,25,90,1,12,1,17,...,0.0,5,2,1,2,13,9,2022-06-14 11:00:00,5,25.0
7700,8,62,7,43,15,105,4,16,0,20,...,0.0,4,1,3,4,13,10,2022-06-14 12:00:00,6,29.0
7701,13,75,10,53,23,128,1,17,0,20,...,0.0,3,1,1,2,13,10,2022-06-14 13:00:00,2,28.0
7702,12,87,8,61,20,148,2,19,0,26,...,0.0,4,0,1,2,12,9,2022-06-14 14:00:00,3,30.0


In [37]:

weather = pd.read_csv('https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/weatherArchiveAndForecast.csv')
weather.ds = pd.to_datetime(weather.ds)
weather = weather.sort_values(by='ds')
weather

,ds,temp,dew,humidity,precip,windspeed,sealevelpressure
0,2020-01-01 00:00:00,0.3,-1.6,87.00,0.0,12.0,99.30
1,2020-01-01 01:00:00,0.2,-1.8,86.00,0.0,12.0,99.29
2,2020-01-01 02:00:00,0.1,-1.5,89.00,0.0,13.0,99.30
3,2020-01-01 03:00:00,-0.2,-1.1,93.00,0.0,12.0,99.30
4,2020-01-01 04:00:00,-0.5,-1.5,93.00,0.0,15.0,99.28
...,...,...,...,...,...,...,...
21501,2022-06-15 19:00:00,26.4,11.6,39.68,0.0,5.4,1019.00
21502,2022-06-15 20:00:00,25.3,12.7,45.53,0.0,4.3,1018.00
21503,2022-06-15 21:00:00,23.3,12.5,50.66,0.0,3.6,1018.00
21504,2022-06-15 22:00:00,21.6,13.1,58.43,0.0,8.3,1018.00


In [38]:

last_timestamp = df.loc[len(df)-1].ds

last_timestamp


Timestamp('2022-06-14 15:00:00')

In [39]:

df = df.merge(weather, on='ds')
df = df.sort_values(by='ds', ascending=True)
df

,Stretcher Pts hrly,Stretcher Pts cum,Ambulatory Pts hrly,Ambulatory Pts cum,y,Total Inflow cum,Ambulances hrly,Ambulances cum,FLS hrly,Adm requests cum,...,Psych pts waiting for admission,ds,Total Pod TBS,Total Vertical TBS,temp,dew,humidity,precip,windspeed,sealevelpressure
0,9,108,5,142,14,250,6,39,0,27,...,0,2021-07-26 19:00:00,10,18.0,25.7,17.6,61.00,0.0,14.0,100.54
1,5,113,6,148,11,261,2,41,0,33,...,0,2021-07-26 20:00:00,12,9.0,23.1,14.6,59.00,0.0,14.0,100.60
2,5,118,2,150,7,268,1,42,0,34,...,0,2021-07-26 21:00:00,8,4.0,21.8,15.5,67.00,0.0,14.0,100.69
3,5,123,8,158,13,281,2,44,0,34,...,0,2021-07-26 22:00:00,9,6.0,21.0,16.0,73.00,0.0,13.0,100.68
4,3,126,7,165,10,291,3,47,0,37,...,0,2021-07-26 23:00:00,10,11.0,20.1,16.4,79.00,0.0,11.0,100.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7686,17,54,8,36,25,90,1,12,1,17,...,9,2022-06-14 11:00:00,5,25.0,22.3,11.9,51.46,0.0,1.8,1013.00
7687,8,62,7,43,15,105,4,16,0,20,...,10,2022-06-14 12:00:00,6,29.0,23.9,12.0,47.03,0.0,3.6,1013.00
7688,13,75,10,53,23,128,1,17,0,20,...,10,2022-06-14 13:00:00,2,28.0,24.0,11.0,44.00,0.0,7.6,1013.00
7689,12,87,8,61,20,148,2,19,0,26,...,9,2022-06-14 14:00:00,3,30.0,23.9,12.0,47.00,0.0,5.4,1012.70


In [40]:

future = pd.concat([df.tail(48),weather[weather.ds>last_timestamp].head(12)])

In [41]:
future

,Stretcher Pts hrly,Stretcher Pts cum,Ambulatory Pts hrly,Ambulatory Pts cum,y,Total Inflow cum,Ambulances hrly,Ambulances cum,FLS hrly,Adm requests cum,...,Psych pts waiting for admission,ds,Total Pod TBS,Total Vertical TBS,temp,dew,humidity,precip,windspeed,sealevelpressure
7643,6.0,39.0,7.0,61.0,13.0,100.0,1.0,12.0,1.0,12.0,...,11.0,2022-06-12 12:00:00,3.0,22.0,23.8,12.0,47.31,0.00,7.6,1011.6
7644,13.0,52.0,2.0,63.0,15.0,115.0,2.0,14.0,0.0,16.0,...,12.0,2022-06-12 13:00:00,1.0,29.0,19.4,14.7,74.62,0.30,5.5,1011.8
7645,10.0,62.0,7.0,70.0,17.0,132.0,2.0,16.0,0.0,15.0,...,12.0,2022-06-12 14:00:00,2.0,30.0,20.9,17.1,79.21,0.00,3.7,1010.7
7646,11.0,73.0,8.0,78.0,19.0,151.0,2.0,18.0,1.0,17.0,...,11.0,2022-06-12 15:00:00,2.0,31.0,21.0,17.9,82.79,0.00,1.9,1010.1
7647,6.0,79.0,4.0,82.0,10.0,161.0,1.0,19.0,0.0,19.0,...,12.0,2022-06-12 16:00:00,4.0,31.0,22.3,15.2,64.48,0.00,1.9,1009.6
7648,7.0,86.0,4.0,86.0,11.0,172.0,3.0,22.0,0.0,23.0,...,12.0,2022-06-12 17:00:00,2.0,25.0,19.5,15.3,77.10,0.00,5.4,1010.0
7649,10.0,96.0,0.0,86.0,10.0,182.0,0.0,22.0,0.0,23.0,...,12.0,2022-06-12 18:00:00,3.0,20.0,18.2,16.3,89.10,0.50,3.6,1009.2
7650,7.0,103.0,2.0,88.0,9.0,191.0,3.0,25.0,1.0,25.0,...,12.0,2022-06-12 19:00:00,1.0,16.0,18.4,16.2,87.49,0.00,1.8,1009.0
7651,7.0,110.0,3.0,91.0,10.0,201.0,3.0,28.0,0.0,26.0,...,12.0,2022-06-12 20:00:00,2.0,11.0,18.9,15.6,81.63,0.00,1.8,1009.0
7652,7.0,117.0,4.0,95.0,11.0,212.0,0.0,28.0,0.0,28.0,...,12.0,2022-06-12 21:00:00,2.0,14.0,20.3,16.3,77.80,0.00,5.4,1010.0


In [42]:

# future = loaded_model.make_future_dataframe(df, periods=12) # periods=m.n_forecasts, n_historic_predictions=False

forecast = loaded_model.predict(future, raw=True, decompose=False)

start = forecast.values.tolist()[0][0]
forecast_length = len(forecast.values.tolist()[0][1:])

forecast_output = pd.DataFrame(columns=['ds','inflow','timestamp'])
forecast_output['ds'] = pd.date_range(start=start, periods=forecast_length, freq='H')
forecast_output['inflow'] = forecast.values.tolist()[0][1:]
forecast_output['timestamp'] = pd.Timestamp.now().round('S').replace(tzinfo=None)
forecast_output.to_csv('../forecasts/inflow_with_weather.csv', index=False)
forecast_output

WARNING - (NP.forecaster.predict) - Raw forecasts are incompatible with plotting utilities
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 95.0% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 95.0% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.forecaster.__handle_missing_data) - 4 missing dates added.
WARNING - (NP.forecaster.__handle_missing_data) - 4 missing values in column y were detected in total. 
INFO - (NP.forecaster.__handle_missing_data) - 4 NaN values in column y were auto-imputed.
WARNING - (NP.forecaster.__handle_missing_data) - 4 missing values in column Stretcher Pts hrly were detected in total. 
INFO - (NP.forecaster.__handle_missing_data) - 4 NaN values in column Stretcher Pts hrly were auto-imputed.
WARNING - (NP.forecaster.__handle_missing_data) - 4

,ds,inflow,timestamp
0,2022-06-14 12:00:00,20.008987,2022-06-14 20:07:46
1,2022-06-14 13:00:00,24.327162,2022-06-14 20:07:46
2,2022-06-14 14:00:00,22.113440,2022-06-14 20:07:46
3,2022-06-14 15:00:00,22.790991,2022-06-14 20:07:46
4,2022-06-14 16:00:00,20.641260,2022-06-14 20:07:46
5,2022-06-14 17:00:00,17.598457,2022-06-14 20:07:46
6,2022-06-14 18:00:00,14.596188,2022-06-14 20:07:46
7,2022-06-14 19:00:00,17.440479,2022-06-14 20:07:46
8,2022-06-14 20:00:00,15.678611,2022-06-14 20:07:46
9,2022-06-14 21:00:00,14.005450,2022-06-14 20:07:46
